In [1]:
import asyncio
from queue import Queue
from threading import Thread

In [2]:
class aiter_to_iter:
    def __init__(self, it):
        self._it = it
        self._queue = Queue()

    def __iter__(self):
        t = Thread(target=self._worker)
        t.start()
        while True:
            done, value = self._queue.get()
            if done:
                break
            yield value

        if value is not None:
            raise value

        t.join()

    def _worker(self):
        async def enqueue_iter():
            try:
                async for el in self._it:
                    self._queue.put((False, el))
                self._queue.put((True, None))
            except Exception as e:
                self._queue.put((True, e))

        asyncio.run(enqueue_iter())

In [3]:
import time
from random import randint

async def foo():
    for i in range(10):
        n = randint(1, 10) / 10
        await asyncio.sleep(n)
        if i == 5:
            raise TypeError('boo')
        yield n


def sync_foo():
    for i in range(10):
        n = randint(1, 10) / 10
        time.sleep(n)
        if i == 5:
            raise TypeError('boo')
        yield n

In [4]:
for i in sync_foo():
    print(i)

0.8
0.5
0.6
0.2
0.8


TypeError: boo

In [5]:
for i in aiter_to_iter(foo()):
    print(i)

1.0
0.3
0.7
0.2
0.3


TypeError: boo

In [6]:
async def foo():
    yield 1
    yield 2
    yield 3

In [7]:
for el in aiter_to_iter(foo()):
    print(el)

1
2
3
